In [38]:
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [64]:
# read in city_pop.csv 
alt_fuel = pd.read_csv("alt_stations.csv")
alt_fuel.head()

/var/folders/dn/3l1c3sld1b77mm1d_l64vfx40000gn/T/ipykernel_5232/2228962334.py:2: DtypeWarning: Columns (6,16,20,31,33,43,46,52,57,58,60,62,65,67,69,71) have mixed types. Specify dtype option on import or set low_memory=False.
  alt_fuel = pd.read_csv("alt_stations.csv")


,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,Restricted Access,RD Blends,RD Blends (French),RD Blended with Biodiesel,RD Maximum Biodiesel Level,NPS Unit Name,CNG Station Sells Renewable Natural Gas,LNG Station Sells Renewable Natural Gas,Maximum Vehicle Class,EV Workplace Charging
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,MD,NaN
1,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LD,NaN
2,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HD,NaN
3,CNG,Arkansas Oklahoma Gas Corp,2100 S Waldron Rd,NaN,Fort Smith,AR,72903,NaN,479-783-3181,E,...,False,NaN,NaN,NaN,NaN,NaN,False,NaN,MD,NaN
4,CNG,Clean Energy - Logan International Airport,1000 Cottage St Ext,"From Route 1, take the first exit after Callah...",East Boston,MA,2128,NaN,866-809-4869,E,...,False,NaN,NaN,NaN,NaN,NaN,True,NaN,MD,NaN


In [65]:
# remove all columns except Fuel Type Code and City
alt_fuel = alt_fuel[["Fuel Type Code", "City"]]
alt_fuel.head()

,Fuel Type Code,City
0,CNG,Montgomery
1,CNG,Atlanta
2,CNG,Atlanta
3,CNG,Fort Smith
4,CNG,East Boston


In [66]:
# create a new column name called ELEC
alt_fuel["ELEC"] = alt_fuel["Fuel Type Code"] == "ELEC"
alt_fuel.head()

,Fuel Type Code,City,ELEC
0,CNG,Montgomery,False
1,CNG,Atlanta,False
2,CNG,Atlanta,False
3,CNG,Fort Smith,False
4,CNG,East Boston,False


In [67]:
# instead of using False and True, use 0 and 1, 1 is for ELEC
alt_fuel["ELEC"] = alt_fuel["ELEC"].astype(int)
alt_fuel.head()

,Fuel Type Code,City,ELEC
0,CNG,Montgomery,0
1,CNG,Atlanta,0
2,CNG,Atlanta,0
3,CNG,Fort Smith,0
4,CNG,East Boston,0


In [96]:
# create new datafram called ELEC_only
elec_only = alt_fuel[alt_fuel["ELEC"] == 1]
elec_only.head()

,Fuel Type Code,City,ELEC
129,ELEC,Sun Valley,1
130,ELEC,Los Angeles,1
131,ELEC,Los Angeles,1
132,ELEC,Los Angeles,1
133,ELEC,Long Beach,1


In [97]:
# groupby on City and count the number of ELEC
elec_only = elec_only.groupby("City").count()
elec_only.head()

,Fuel Type Code,ELEC
City,,
8900 E. Buckeye Ave.,1,1
AIKEN,1,1
ALBERT LEA,1,1
ALGONA,1,1
ALLENTOWN,1,1


In [99]:
# value count on ELEC
elec_only["ELEC"].value_counts()

1      2284
2      1049
3       584
4       386
5       309
       ... 
80        1
131       1
224       1
156       1
132       1
Name: ELEC, Length: 172, dtype: int64

In [100]:
# reset index for ELEC only
elec_only = elec_only.reset_index()
elec_only.head()

,City,Fuel Type Code,ELEC
0,8900 E. Buckeye Ave.,1,1
1,AIKEN,1,1
2,ALBERT LEA,1,1
3,ALGONA,1,1
4,ALLENTOWN,1,1


In [101]:
# drop Fuel Type Code 
elec_only = elec_only.drop(columns=["Fuel Type Code"])
elec_only.head()

,City,ELEC
0,8900 E. Buckeye Ave.,1
1,AIKEN,1
2,ALBERT LEA,1
3,ALGONA,1
4,ALLENTOWN,1


In [102]:
# read in city_pop.csv
city_pop = pd.read_csv("city_pop.csv")
city_pop.head()

,City,State,Total Population
0,Auburn,Alabama,80009
1,Birmingham,Alabama,196353
2,Dothan,Alabama,70524
3,Hoover,Alabama,92427
4,Huntsville,Alabama,222363


In [103]:
# remove all rows with Puerto Rico in the State column
city_pop = city_pop[city_pop.State != "Puerto Rico"]
city_pop.head()

,City,State,Total Population
0,Auburn,Alabama,80009
1,Birmingham,Alabama,196353
2,Dothan,Alabama,70524
3,Hoover,Alabama,92427
4,Huntsville,Alabama,222363


In [104]:
# abbreviate State column
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
'Colorado': 'CO',
'Connecticut': 'CT',
'District of Columbia': 'DC',
'Delaware': 'DE',
'Florida': 'FL',
'Georgia': 'GA',
'Hawaii': 'HI',
'Idaho': 'ID',
'Illinois': 'IL',
'Indiana': 'IN',
'Iowa': 'IA',
'Kansas': 'KS',
'Kentucky': 'KY',
'Louisiana': 'LA',
'Maine': 'ME',
'Maryland': 'MD',
'Massachusetts': 'MA',
'Michigan': 'MI',
'Minnesota': 'MN',
'Mississippi': 'MS',
'Missouri': 'MO',
'Montana': 'MT',
'Nebraska': 'NE',
'Nevada': 'NV',
'New Hampshire': 'NH',
'New Jersey': 'NJ',
'New Mexico': 'NM',
'New York': 'NY',
'North Carolina': 'NC',
'North Dakota': 'ND',
'Ohio': 'OH',
'Oklahoma': 'OK',
'Oregon': 'OR',
'Pennsylvania': 'PA',
'Rhode Island': 'RI',
'South Carolina': 'SC',
'South Dakota': 'SD',
'Tennessee': 'TN',
'Texas': 'TX',
'Utah': 'UT',
'Vermont': 'VT',
'Virginia': 'VA',
'Washington': 'WA',
'West Virginia': 'WV',
'Wisconsin': 'WI',
'Wyoming': 'WY'}

city_pop['State'] = city_pop['State'].replace(state_abbreviations)


city_pop.head()


,City,State,Total Population
0,Auburn,AL,80009
1,Birmingham,AL,196353
2,Dothan,AL,70524
3,Hoover,AL,92427
4,Huntsville,AL,222363


In [105]:
# do a unique for the State column
city_pop["State"].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'WA', 'WI', 'WY'], dtype=object)

In [106]:
# read in city_income.csv
city_income = pd.read_csv("city_income.csv")
city_income.head()

,City,State,Median Income (16 and over)
0,Auburn,Alabama,31442
1,Birmingham,Alabama,34547
2,Dothan,Alabama,37493
3,Hoover,Alabama,58486
4,Huntsville,Alabama,42391


In [107]:
# row count
city_income.count()

City                           646
State                          646
Median Income (16 and over)    646
dtype: int64

In [108]:
# remove all rows with Puerto Rico in the State column
city_income = city_income[city_income.State != "Puerto Rico"]
city_income.head()

,City,State,Median Income (16 and over)
0,Auburn,Alabama,31442
1,Birmingham,Alabama,34547
2,Dothan,Alabama,37493
3,Hoover,Alabama,58486
4,Huntsville,Alabama,42391


In [109]:
# row count
city_income.count()

City                           640
State                          640
Median Income (16 and over)    640
dtype: int64

In [110]:
# abbreviate State column
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
'Colorado': 'CO',
'Connecticut': 'CT',
'District of Columbia': 'DC',
'Delaware': 'DE',
'Florida': 'FL',
'Georgia': 'GA',
'Hawaii': 'HI',
'Idaho': 'ID',
'Illinois': 'IL',
'Indiana': 'IN',
'Iowa': 'IA',
'Kansas': 'KS',
'Kentucky': 'KY',
'Louisiana': 'LA',
'Maine': 'ME',
'Maryland': 'MD',
'Massachusetts': 'MA',
'Michigan': 'MI',
'Minnesota': 'MN',
'Mississippi': 'MS',
'Missouri': 'MO',
'Montana': 'MT',
'Nebraska': 'NE',
'Nevada': 'NV',
'New Hampshire': 'NH',
'New Jersey': 'NJ',
'New Mexico': 'NM',
'New York': 'NY',
'North Carolina': 'NC',
'North Dakota': 'ND',
'Ohio': 'OH',
'Oklahoma': 'OK',
'Oregon': 'OR',
'Pennsylvania': 'PA',
'Rhode Island': 'RI',
'South Carolina': 'SC',
'South Dakota': 'SD',
'Tennessee': 'TN',
'Texas': 'TX',
'Utah': 'UT',
'Vermont': 'VT',
'Virginia': 'VA',
'Washington': 'WA',
'West Virginia': 'WV',
'Wisconsin': 'WI',
'Wyoming': 'WY'}

city_income['State'] = city_income['State'].replace(state_abbreviations)


city_income.head()

,City,State,Median Income (16 and over)
0,Auburn,AL,31442
1,Birmingham,AL,34547
2,Dothan,AL,37493
3,Hoover,AL,58486
4,Huntsville,AL,42391


In [111]:
# remove State column from city_income
city_income = city_income.drop(columns=["State"])
city_income.head()

,City,Median Income (16 and over)
0,Auburn,31442
1,Birmingham,34547
2,Dothan,37493
3,Hoover,58486
4,Huntsville,42391


In [113]:
# merge alt_stations and city_income on City column
alt_fuel_income = pd.merge(elec_only, city_income, on="City")
alt_fuel_income.head(10)

,City,ELEC,Median Income (16 and over)
0,Abilene,6,35115
1,Akron,44,36969
2,Alameda,49,82773
3,Albany,212,26766
4,Albany,212,39005
5,Albuquerque,79,41984
6,Alexandria,104,79029
7,Alhambra,10,51560
8,Allen,9,66616
9,Allentown,48,35493


In [115]:
# merge alt_fuel_income and city_pop on City column
alt_updated = pd.merge(alt_fuel_income, city_pop, on="City")
alt_updated.head(25)

,City,ELEC,Median Income (16 and over),State,Total Population
0,Abilene,6,35115,TX,128149
1,Akron,44,36969,OH,188498
2,Alameda,49,82773,CA,76039
3,Albany,212,26766,GA,67922
4,Albany,212,26766,NY,100832
5,Albany,212,39005,GA,67922
6,Albany,212,39005,NY,100832
7,Albuquerque,79,41984,NM,561006
8,Alexandria,104,79029,VA,155525
9,Alhambra,10,51560,CA,80264


In [116]:
# row count
alt_updated.count()

City                           621
ELEC                           621
Median Income (16 and over)    621
State                          621
Total Population               621
dtype: int64

In [117]:
# remove the State column
alt_updated = alt_updated.drop(columns=["State"])
alt_updated.head()

,City,ELEC,Median Income (16 and over),Total Population
0,Abilene,6,35115,128149
1,Akron,44,36969,188498
2,Alameda,49,82773,76039
3,Albany,212,26766,67922
4,Albany,212,26766,100832


In [118]:
# remove city column
alt_updated = alt_updated.drop(columns=["City"])
alt_updated.head()

,ELEC,Median Income (16 and over),Total Population
0,6,35115,128149
1,44,36969,188498
2,49,82773,76039
3,212,26766,67922
4,212,26766,100832


In [120]:
# row count
alt_updated.count()

ELEC                           621
Median Income (16 and over)    621
Total Population               621
dtype: int64

In [121]:
# describe the data
alt_updated.describe()


,ELEC,Median Income (16 and over),Total Population
count,621.000000,621.000000,6.210000e+02
mean,72.956522,47048.003221,1.948571e+05
std,119.183687,14520.746232,4.257633e+05
min,1.000000,21600.000000,6.399100e+04
25%,16.000000,37932.000000,8.026400e+04
50%,38.000000,42882.000000,1.073250e+05
75%,79.000000,52250.000000,1.700620e+05
max,1766.000000,128496.000000,8.335897e+06


In [122]:
from sklearn.linear_model import LinearRegression


In [123]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = alt_updated["ELEC"]

# Separate the X variable, the features
X = alt_updated.drop(columns="ELEC")

In [139]:
# Create a model with scikit-learn
model = LinearRegression()

In [140]:
# Fit the data into the model
model.fit(X, y)

LinearRegression()

In [141]:
# Display the slope
print(f"Model's slope: {model.coef_}")

# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's slope: [0.00118374 0.00015083]
Model's y-intercept: -12.126361536467684
Model's formula: y = -12.126361536467684 + 0.0011837388514330447X


In [149]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import pickle


In [150]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [151]:
# Compute the metrics for the linear regression model
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
mae = mean_absolute_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The mean absolute error is {mae}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.31027081720660143.
The r2 is 0.31027081720660143.
The mean squared error is 9781.654665914151.
The mean absolute error is 53.68138356347275.
The root mean squared error is 98.90224803266179.
The standard deviation is 119.08768759487263.


In [152]:
# import pickle
# # Save the model
projectfile = 'model.sav'

In [153]:
# save the model projectfile
pickle.dump(model, open(projectfile, 'wb'))

In [155]:
# create new data frame
df = pd.DataFrame({"Median Income (16 and over)": [50000], "Total Population": [100000]})

predictions = model.predict(df)

print(predictions)

[62.14360283]
